In [1]:
%pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import importlib
from sklearn.metrics import root_mean_squared_error

import utils

importlib.reload(utils)

from utils import load_data

In [3]:
def reduce_mem_usage(df):
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [4]:
data = load_data(prepare=False)
data = data.sort_values(by=["id","day_num"])
data.loc[:, 'lag'] = data.groupby('id')['value'].shift(1)

In [5]:
data[data["lag"].isna()]["day_num"].describe()

count    30490.0
mean         1.0
std          0.0
min          1.0
25%          1.0
50%          1.0
75%          1.0
max          1.0
Name: day_num, dtype: float64

In [6]:
data.groupby("id").size().describe()

count    30490.0
mean      1941.0
std          0.0
min       1941.0
25%       1941.0
50%       1941.0
75%       1941.0
max       1941.0
dtype: float64

In [7]:
data["train"] = data["day_num"]<=1913

In [8]:
data.head(10)

,id,item_id,dept_id,cat_id,store_id,state_id,d,value,day_num,date,...,year,event_name_1,event_type_1,event_name_2,event_type_2,snap,sell_price,total_volume,train,lag
1612,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1,3,1,2011-01-29,...,2011,None,None,None,None,0,2.0,6.0,True,NaN
32102,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_2,0,2,2011-01-30,...,2011,None,None,None,None,0,2.0,0.0,True,3.0
62592,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_3,0,3,2011-01-31,...,2011,None,None,None,None,0,2.0,0.0,True,0.0
93082,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_4,1,4,2011-02-01,...,2011,None,None,None,None,1,2.0,2.0,True,0.0
123572,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_5,4,5,2011-02-02,...,2011,None,None,None,None,1,2.0,8.0,True,1.0
154062,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_6,2,6,2011-02-03,...,2011,None,None,None,None,1,2.0,4.0,True,4.0
184552,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_7,0,7,2011-02-04,...,2011,None,None,None,None,1,2.0,0.0,True,2.0
215042,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_8,2,8,2011-02-05,...,2011,None,None,None,None,1,2.0,4.0,True,0.0
245532,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_9,0,9,2011-02-06,...,2011,SuperBowl,Sporting,None,None,1,2.0,0.0,True,2.0
276022,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_10,0,10,2011-02-07,...,2011,None,None,None,None,1,2.0,0.0,True,0.0


In [9]:
data["events"] = data["event_name_1"].notna()+data["event_name_2"].notna()

## ETS

In [10]:
from statsforecast import StatsForecast
from statsforecast.models import (
    AutoETS,SeasonalNaive
)
models = [
    AutoETS()
]


/home/jan/m5-challenge/.venv/lib64/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
sf = StatsForecast( 
    models=models,
    freq="M", 
    fallback_model = SeasonalNaive(season_length=7),
    n_jobs=-1,
)

In [28]:
train = data[data['train']].dropna(subset="sell_price")
train = train[train["day_num"]>1]

test = data[~data['train']]


In [13]:
train.head()

Y_df = train.rename(columns = {"id":"unique_id","date":"ds","value":"y"}).filter(["unique_id","ds","y"])

In [14]:
fc = sf.forecast(df=Y_df, h=3)

/home/jan/m5-challenge/.venv/lib64/python3.12/site-packages/utilsforecast/processing.py:384: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  freq = pd.tseries.frequencies.to_offset(freq)
/home/jan/m5-challenge/.venv/lib64/python3.12/site-packages/utilsforecast/processing.py:440: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  freq = pd.tseries.frequencies.to_offset(freq)


In [16]:
train["squared_diff"] = (train["value"] - train["lag"]) **2

mse_train = (
    train.groupby("id")["squared_diff"]
        .mean()
).rename("mse_train")



In [33]:
weights = (
    train[train["day_num"]>(1914-28)]
        .groupby("id")["total_volume"]
        .sum()
)

weights = weights/weights.sum()

In [30]:
test = test.merge(fc.rename(columns={"unique_id":"id","ds":"date"}),on=["id","date"])

In [31]:

test["squared_diff"] = (test["value"] - test["AutoETS"])**2

mse_test = (
    test
        .groupby("id")["squared_diff"]
        .mean()
).rename("mse_test")



In [34]:
t = (
    pd.concat([mse_test, mse_train, weights], axis=1)
)

result = (
    t["total_volume"] * np.sqrt( (t["mse_test"] / t["mse_train"]) )
).sum()

In [36]:
result

np.float64(0.7086701968542939)

In [ ]:
data[["lag","value","events"]].corr()

,lag,value,events
lag,1.000000,0.767991,0.006919
value,0.767991,1.000000,-0.003660
events,0.006919,-0.003660,1.000000


In [ ]:
dummies = pd.get_dummies(data[["event_name_1","event_type_1","event_name_2","event_type_2"]])

dummie_columns = dummies.columns

In [12]:
data = pd.concat([data,dummies],axis=1)

In [13]:
del dummies

In [14]:
data['sell_price_filled'] = data.groupby('id')['sell_price'].ffill().bfill()

In [15]:
train = data[data['train']].dropna(subset="sell_price")
train = train[train["day_num"]>1]

test = data[~data['train']]

In [16]:
import category_encoders as ce

categories = ["event_name_1", "event_type_1", "event_name_2", "event_type_2", "cat_id", "state_id","wday","snap"]
target_enc = ce.TargetEncoder(cols=categories)
encoded_train = target_enc.fit_transform(train[categories], train['value'])
encoded_train = encoded_train.rename(columns=lambda col: f"{col}_encoded")

encoded_columns = encoded_train.columns

In [17]:
train = pd.concat([train,encoded_train],axis=1)

In [18]:
encoded_test = target_enc.fit_transform(test[categories], test['value'])
encoded_test = encoded_test.rename(columns=lambda col: f"{col}_encoded")

test = pd.concat([test,encoded_test],axis=1)

In [19]:
del encoded_train, encoded_test

In [20]:
features = list(encoded_columns)+["day_num", "wday", "month", "year", "snap", "lag", "sell_price"]

In [21]:
X_train = train[features]
y_train = train['value']


X_test = test[features]
y_test = test['value']

In [22]:
import lightgbm as lgb


In [23]:
gbm = lgb.LGBMRegressor(objective ='poisson' )

gbm.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric='l2'
        )

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.617384 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 690
[LightGBM] [Info] Number of data points in the train set: 46017025, number of used features: 15
[LightGBM] [Info] Start training from score 0.355521


LGBMRegressor(objective='poisson')

In [24]:
from sklearn import linear_model
clf = linear_model.PoissonRegressor(max_iter=200)
clf.fit(X_train, y_train)

/home/jan/.local/lib/python3.13/site-packages/sklearn/linear_model/_glm/glm.py:283: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res)


PoissonRegressor(max_iter=200)

In [25]:
clf.score(X_train, y_train)

0.20317627587385623

In [26]:
train["id"].nunique()

30490

In [27]:
def make_prediciton(regressor):
    predictions = []

    for day in test["day_num"].unique():
        # Combine the current lags and additional features
        t = test.loc[test["day_num"]==day,:]    
        
        # Make the prediction

        if type(regressor) == "lightgbm.sklearn.LGBMRegressor":
            next_pred = gbm.predict(X_train, num_iteration=gbm.best_iteration_)
        else:
            next_pred = regressor.predict(t[features])

        predictions.append(next_pred)

        if day < 1941:
            test.loc[ (test["day_num"]==day+1) ,"lag"] = next_pred

    print("Recursive predictions:", predictions)
    return( np.array(predictions).T.flatten())

In [28]:
test["fc_lin"] = make_prediciton(clf)

Recursive predictions: [array([1.69626705, 1.79911731, 1.69626705, ..., 0.70211335, 0.70211335,
       0.70211335]), array([1.67729134, 1.67983201, 1.67729134, ..., 0.68417557, 0.68417557,
       0.68417557]), array([1.61912508, 1.61918562, 1.61912508, ..., 0.66045935, 0.66045935,
       0.66045935]), array([1.563018  , 1.56301939, 1.563018  , ..., 0.63789596, 0.63789596,
       0.63789596]), array([1.51348501, 1.51348504, 1.51348501, ..., 0.61798564, 0.61798564,
       0.61798564]), array([1.87055792, 1.87055792, 1.87055792, ..., 0.76411856, 0.76411856,
       0.76411856]), array([1.86817242, 1.86817242, 1.86817242, ..., 0.737196  , 0.737196  ,
       0.737196  ]), array([1.78840547, 1.78840547, 1.78840547, ..., 0.72803217, 0.72803217,
       0.72803217]), array([1.72277705, 1.72277705, 1.72277705, ..., 0.70204492, 0.70204492,
       0.70204492]), array([1.66189201, 1.66189201, 1.66189201, ..., 0.65662376, 0.65662376,
       0.65662376]), array([1.60306968, 1.60306968, 1.60306968, ...

In [29]:
test["fc_gbm"] = make_prediciton(gbm)

Recursive predictions: [array([0.59531442, 2.92139259, 0.59531442, ..., 0.34289853, 0.34289853,
       0.34289853]), array([1.15022544, 1.94083801, 1.15022544, ..., 0.63154029, 0.63154029,
       0.63154029]), array([1.1603609 , 1.50652159, 1.1603609 , ..., 0.63548392, 0.63548392,
       0.63548392]), array([1.1603609 , 1.50652159, 1.1603609 , ..., 0.63548392, 0.63548392,
       0.63548392]), array([1.35621272, 1.79634885, 1.35621272, ..., 0.75118781, 0.75118781,
       0.75118781]), array([1.47847183, 1.9303135 , 1.47847183, ..., 0.90683424, 0.90683424,
       0.90683424]), array([1.44555639, 1.85379466, 1.44555639, ..., 0.84128804, 0.84128804,
       0.84128804]), array([1.36548365, 1.8019651 , 1.36548365, ..., 0.7514616 , 0.7514616 ,
       0.7514616 ]), array([1.18146217, 1.56642825, 1.18146217, ..., 0.63685999, 0.63685999,
       0.63685999]), array([1.1859428 , 1.58030176, 1.1859428 , ..., 0.63548392, 0.63548392,
       0.63548392]), array([1.1859428 , 1.58030176, 1.1859428 , ...

In [30]:
test["fc_lin"].describe()

count    853720.000000
mean          1.178574
std           0.552996
min           0.003293
25%           0.750471
50%           1.201733
75%           1.613541
max          12.960583
Name: fc_lin, dtype: float64

In [31]:
test["fc_gbm"].describe()

count    853720.000000
mean          1.501582
std           1.875773
min           0.268423
25%           0.765961
50%           1.066229
75%           1.506522
max         161.913563
Name: fc_gbm, dtype: float64

In [32]:
root_mean_squared_error(test["value"],test["fc_lin"])

3.5622501105064974

In [33]:
root_mean_squared_error(test["value"],test["fc_gbm"])

3.11778099675722

In [38]:
mse_train.describe()

count    30490.000000
mean         7.873469
std         50.214720
min          0.025105
25%          0.678340
50%          1.660325
75%          4.230648
max       4929.767782
Name: mse_train, dtype: float64

In [39]:
test["squared_diff"] = (test["value"] - test["fc_gbm"])**2

mse_test = (
    test
        .groupby("id")["squared_diff"]
        .mean()
).rename("mse_test")



In [40]:
mse_test.describe()

count    30490.000000
mean         9.720558
std         69.151729
min          0.258075
25%          0.670179
50%          1.315643
75%          3.211346
max       3442.741306
Name: mse_test, dtype: float64

In [42]:
t = (
    pd.concat([mse_test, mse_train, weights], axis=1)
)

result = (
    t["total_volume"] * np.sqrt( (t["mse_test"] / t["mse_train"]) )
).sum()